Assignment 3
Hanson, Hanxiang Zhang, 300100018

Data preparation:
I imported the data from the files.
There are two ways of concatenating the input start and goal images.
Method 1:
Concatenate the pair of start and goal images horizontally into a new image with 3 channels (Blue, Green, Red). The image size is 128 by 64. The array named train_img_rot is for training rotation dataset, train_img_mix for training mix dataset, test_img_rot for testing rotation dataset, test_img_mix for testing mix dataset. Validation is included in the training dataset.
Method 2:
keep the original data format, where each image pair has 6 dimensions, where the first 3 channels (Blue, Green, Red) belong to the start image and the second 3 channels belong to the goal image. The image size is 64 by 64. The array named is for train_images for training rotation dataset, train_images_mix for training mix dataset, test_images for testing rotation dataset, test_images_mix for testing mix dataset.
Which method to choose?
I selected method 2 after I tried many architectures and the neural network could not learn, i.e. decrease the training error and validation error when I used data prepared with method 1.
For example, I used the same architecture with 7 conv layers with 64 neurons in each layer for both data prepared by method 1 and 2 to run the training process, and the training Mean Squared Error for method 1 data is around 0.5978 after 100 epoches, which is close to the initial MSE which is 0.61. Meanwhile the validation MSE goes up to 0.7128 from 0.58 which is quite bad. However, for method 2 data the training MSE graduately decreases and ends up with 0.0050, while the initial MSE is 0.60.
Meanwhile, the validation MSE decreases incrementally and ends up with 0.0310, while the initial valid MSE is 0.59.

And the train_flows is the optical flow for rotational training data, which has removed the last two channels because they are the negative of the first two channels respectively. Where the four channels are [flow(x); flow(y); flow(-x); flow(-y)]. Likewise for the testing rotation optical flow, named test_flows, and training and testing mixed optical flow, named train_flows_mix and test_flows_mix respectively.

After importing the dataset, I shuffled the order of training data and testing data respectively using sklearn.utils.shuffle.

Dataset Review:
There are 1860 rotation image pairs for training, 444 rotation image pairs for testing.
And there are 1860 mixed image pairs for training, 444 mixed image pairs for testing.
Validation is in the training images. The validation rate is set to be 0.11. As a result, there are 1655 image pairs for training, and 205 for validation.

In [1]:
# Common imports
import os
import sys
print(sys.executable)
import skimage 
import numpy as np
from sklearn.utils import shuffle

# to make this notebook's output stable across runs
np.random.seed(42)

# The data directory
dire = os.getcwd()
print(dire)
PROJECT_ROOT_DIR = "C:/Users/hanson/OneDrive/Grad/CNN in CV/data/" #Directory for image data

train_images = np.load(os.path.join(PROJECT_ROOT_DIR, 'rot_images_train.npy'))
train_flows = np.load(os.path.join(PROJECT_ROOT_DIR, 'rot_flows_train.npy'))
train_images, train_flows = shuffle(train_images, train_flows) #Shuffle the data

test_images = np.load(os.path.join(PROJECT_ROOT_DIR, 'rot_images_test.npy'))
test_flows = np.load(os.path.join(PROJECT_ROOT_DIR, 'rot_flows_test.npy'))
test_images, test_flows = shuffle(test_images, test_flows) #Shuffle the data

train_images_mix = np.load(os.path.join(PROJECT_ROOT_DIR, 'mix_images_train.npy'))
train_flows_mix = np.load(os.path.join(PROJECT_ROOT_DIR, 'mix_flows_train.npy'))
train_images_mix, train_flows_mix = shuffle(train_images_mix, train_flows_mix) #Shuffle the data

test_images_mix = np.load(os.path.join(PROJECT_ROOT_DIR, 'mix_images_test.npy'))
test_flows_mix = np.load(os.path.join(PROJECT_ROOT_DIR, 'mix_flows_test.npy'))
test_images_mix, test_flows_mix = shuffle(test_images_mix, test_flows_mix) #Shuffle the data

print('Size of training images: {0} x {1} x {2} x {3}\n'.format(*train_images.shape))
#print('Size of flow results: {0} x {1} x {2} x {3}\n'.format(*train_flows.shape))
print('Size of testing images: {0} x {1} x {2} x {3}\n'.format(*test_images.shape))
#print('Size of flow results: {0} x {1} x {2} x {3}\n'.format(*test_flows.shape))
print('Size of training images mix: {0} x {1} x {2} x {3}\n'.format(*train_images_mix.shape))
#print('Size of flow results: {0} x {1} x {2} x {3}\n'.format(*train_flows_mix.shape))
print('Size of testing images mix: {0} x {1} x {2} x {3}\n'.format(*test_images_mix.shape))
#print('Size of flow results: {0} x {1} x {2} x {3}\n'.format(*test_flows_mix.shape))
train_flows=train_flows[:,:2,:,:] #slicing
#train_flows=train_flows.reshape((-1,2,64,64))
test_flows=test_flows[:,:2,:,:]
train_flows_mix=train_flows_mix[:,:2,:,:]
test_flows_mix=test_flows_mix[:,:2,:,:]
print('Size of train flow results: {0} x {1} x {2} x {3}\n'.format(*train_flows.shape))
print('Size of test flow results: {0} x {1} x {2} x {3}\n'.format(*test_flows.shape))
##print('Size of flow results: {0} x {1} x {2} x {3}\n'.format(*train_flows_mix.shape))
#print('Size of flow results: {0} x {1} x {2} x {3}\n'.format(*test_flows_mix.shape))

def concatenate(images):
    images_start=images[:,:3,:,:]
    images_goal=images[:,3:,:,:]
 #   print('Size of start images: {0} x {1} x {2} x {3}\n'.format(*images_start.shape))
 #   print('Size of goal images: {0} x {1} x {2} x {3}\n'.format(*images_goal.shape))
    con_images=np.concatenate((np.array(images_start),np.array(images_goal)), axis=2)
#    print('Size of combined images: {0} x {1} x {2} x {3}\n'.format(*con_images.shape))
    return con_images
    
train_img_rot=concatenate(train_images);
print('TRAIN ROT Size of combined images: {0} x {1} x {2} x {3}\n'.format(*train_img_rot.shape))
test_img_rot=concatenate(test_images);
print('TEST ROT Size of combined images: {0} x {1} x {2} x {3}\n'.format(*test_img_rot.shape))

train_img_mix=concatenate(train_images_mix);
#print('TRAIN MIX Size of combined images: {0} x {1} x {2} x {3}\n'.format(*train_img_mix.shape))
test_img_mix=concatenate(test_images_mix);
#print('TEST MIX Size of combined images: {0} x {1} x {2} x {3}\n'.format(*test_img_mix.shape))


C:\Users\hanson\Anaconda3\python.exe
C:\Users\hanson\Downloads
Size of training images: 1860 x 6 x 64 x 64

Size of testing images: 444 x 6 x 64 x 64

Size of training images mix: 1860 x 6 x 64 x 64

Size of testing images mix: 444 x 6 x 64 x 64

Size of train flow results: 1860 x 2 x 64 x 64

Size of test flow results: 444 x 2 x 64 x 64

TRAIN ROT Size of combined images: 1860 x 3 x 128 x 64

TEST ROT Size of combined images: 444 x 3 x 128 x 64



As mentioned before, the input to the network is determined to be the 6 channel images of 64 by 64, which is the original data format. This is because the other data format where image pairs are concatenated into 128 by 64 does not work for my CNN.

I used 7 hidden layers to create the CNN network for regression using Sequential API. In each hidden layer, there is one convolutional layer with activation function followed. The model is shown as below.

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 64)        9664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        4160      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        102464    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 64)        102464    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 64, 64)        102464    
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 2, 64, 64)         1154      
Total params: 396,226
Trainable params: 396,226
Non-trainable params: 0
_________________________________________________________________


The number of filters is 64 for each layer. It has a filter size that is either (5, 5), (3, 3) or (1, 1). I changed some kernal size to (3, 3) but the performance seems to be similar.

The activation function is Rectified Linear Unit, which is shown to perform best by my experiment. I tried Sigmoid and Tanh as well, but neither of them could make the CNN learn since MSE does not go down throughout the 100 training epoches.

And the maximum pooling layer were initially followed, but finally removed because this is regression rather than classification. Max pooling in this task seems to have trivial influence on the performance.

I also tried different learning rate for optimizers such as 0.005. However, the default learning rate which is 0.001 is the only learning rate that works well. Optimizers such as SGD, Adam, Nadam and Adagrad are also tried. Adam is the best and selected. While optimizer SGD makes the MSE decrease slower than Adam, but at least the MSE is decreasing for both training and validation. Optimizer Nadam does not work at all, training MSE is constantly 0.6610 and validation MSE is always about 0.5277. Optimizer Adagrad makes the MSE decrease faster in the beginning compared to later epoches. The speed of decreasing of MSE become slower along with the increase of number of epoches. This is because Adagrad use adaptive learning rate, which makes the changes sligher for parameters that are changed more frequently. As a result, the learning of CNN becomes slower than Adam optimizer.

The loss function is Mean Squared Error, where Loss = square(y_true - y_pred). Because the optical flow has two channels indicating both x and y axis, based on the equation, Mean Squared Error is able to take the two components of the flow into account.

The performance measurement I used is Mean Squared Error. Apparently we cannot use accuracy, recall and precision because this is a regression task. Another metrics for regression is Max Error, which is not suitable since it measures the maximum residual error. And it does not support multi-output for now. Besides, Mean absolute error should be okay for this task, but it is not selected due to the fact that it is more suitable for data where there're a bit outliers. And Mean Squared Logarithmic Error cannot be used when targets contain negative values.
https://scikit-learn.org/stable/modules/model_evaluation.html#median-absolute-error

In [2]:
### from keras.models import Sequential
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt
#from keras.layers import Conv2D, MaxPooling2D, MaxPooling3D, MaxPooling1D
from keras.layers import Activation, Dropout, Flatten
from keras.layers import Dense
import tensorflow as tf

#For the RTX 2060 GPU to work in Tensorflow-gpu 2.0, setting the memory growth.
import tensorflow.keras
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, MaxPooling3D, UpSampling3D,Conv2DTranspose, Conv3DTranspose

model = Sequential()

#model.add(Conv2D(64, kernel_size=(5, 5), activation='relu', padding='same', data_format='channels_first', input_shape=(3,128,64)))
model.add(Conv2D(64, kernel_size=(5, 5), activation='relu', padding='same', data_format='channels_first', input_shape=(6,64,64)))
#model.add(MaxPooling2D(pool_size=(2,1)))

model.add(Conv2D(64, (1, 1), activation='relu', padding='same', data_format='channels_first'))

model.add(Conv2D(64, (5, 5), activation='relu', padding='same', data_format='channels_first'))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first'))

model.add(Conv2D(64, (5, 5), activation='relu', padding='same', data_format='channels_first'))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first'))

model.add(Conv2D(64, (5, 5), activation='relu', padding='same', data_format='channels_first'))

model.add(Conv2D(2, (3, 3), activation='linear', padding='same', data_format='channels_first'))

sgd = tensorflow.keras.optimizers.SGD(lr=0.001)
adam = tensorflow.keras.optimizers.Adam(lr=0.001)
nadam = tensorflow.keras.optimizers.Nadam(lr=0.001)
adagrad = tensorflow.keras.optimizers.Adagrad(lr=0.001)
model.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer='adam',
              metrics=['mean_squared_error'])
print(model.summary())

model1=model #Copy the model for mixed dataset

Using TensorFlow backend.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 64)        9664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        4160      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        102464    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 64)        102464    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 64, 64)        1

And I trained it on the rotation training set. 
Because the order in dataset, the weights and biases in CNN are randomly initialized each time, there is a possibility that sometimes the MSE of the model stays at 0.6621 and does not change. If so, run the program again.
Performance for training and validation dataset:
The learning of CNN seems really good, the training MSE decreases gradually from 0.6145 to 0.0037 after 100 epochs, while the validation MSE decreases gradually from 0.6250 to 0.0055. Part of the training & validation result is shown below.

Train on 1655 samples, validate on 205 samples
Epoch 1/100
1655/1655 [==============================] - 4s 3ms/sample - loss: 0.6145 - mean_squared_error: 0.6145 - val_loss: 0.6250 - val_mean_squared_error: 0.6250
Epoch 2/100
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.5011 - mean_squared_error: 0.5011 - val_loss: 0.3734 - val_mean_squared_error: 0.3734
...

Epoch 100/100
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.0037 - mean_squared_error: 0.0037 - val_loss: 0.0055 - val_mean_squared_error: 0.0055

In [3]:
batchSize = 10
nEpochs = 20 #100

#history = model.fit(train_img_rot, train_flows, batch_size=batchSize, epochs=nEpochs, verbose=1,validation_split=0.11)
history = model.fit(train_images, train_flows, batch_size=batchSize, epochs=nEpochs, verbose=1,validation_split=0.11)

Train on 1655 samples, validate on 205 samples
Epoch 1/20
1655/1655 [==============================] - 6s 4ms/sample - loss: 0.5355 - mean_squared_error: 0.5355 - val_loss: 0.3622 - val_mean_squared_error: 0.3622
Epoch 2/20
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.2536 - mean_squared_error: 0.2536 - val_loss: 0.2057 - val_mean_squared_error: 0.2057
Epoch 3/20
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.1845 - mean_squared_error: 0.1845 - val_loss: 0.1537 - val_mean_squared_error: 0.1537
Epoch 4/20
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.1364 - mean_squared_error: 0.1364 - val_loss: 0.1250 - val_mean_squared_error: 0.1250
Epoch 5/20
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.1083 - mean_squared_error: 0.1083 - val_loss: 0.0975 - val_mean_squared_error: 0.0975
Epoch 6/20
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.0878 - mean_squared_error: 0.0878 - val_los

Evaluate it on the rotation test set:
After I got the best CNN model, I finalize the model and test it on the rotation test set.
The performance on the test set is very good, which is lower than the training set.
Mean Squared Error for Test Set is: 0.0056, which is a bit higher than the valiadation MSE by 0.0001. And test MSE is higher than the training MSE by 0.0019.
This is a model that shows a little bit overfitting because the model perform better for training dataset. However, because the MSE for test set is already very low and is basically equivalent to MSE for valdiation set, this model is still good.

In [4]:
from sklearn.metrics import mean_squared_error

test_flow_result = model.predict(test_images)
#Reshape to make the flow array two dimensional so that it can use sklearn MSE function
test_flowsa=test_flows.reshape(test_flows.shape[0],8192) #8192=2*64*64
print(test_flows.shape[0])
test_flow_resulta=test_flow_result.reshape(test_flows.shape[0],8192)

mse=mean_squared_error(test_flowsa, test_flow_resulta)
#msle=mean_squared_log_error(train_flowsa, train_flow_resulta)
#Mean Squared Logarithmic Error cannot be used when targets contain negative values
print("Mean Squared Error for Test Set is: %.4f"%mse)
#print(str(msle))
#print(str(medae))

444
Mean Squared Error for Test Set is: 0.0213


4. Transfer Learning
concatenate the input start and goal images:
In order to adapt the input to VGG16, I changed the format of the input array from 1860x6x64x64, where the six channels are [Blue1; Green1;Red1;Blue2; Green2;Red2] to 1860x128x64x3, where three channels are Red, Green and Blue.

Because channels are the last dimension in data format in VGG, I changed the order of dimensions in flow data as well, from 2x64x64 to 64x64x2.

In [5]:
def concate_tl(im_ar):
    result=np.zeros((im_ar.shape[0], 128, 64, 3))
    #print(im_ar[66,:,13,20])

    im_start=im_ar[:,:3,:,:]
    im_start_rgb=np.zeros(im_start.shape)
    #print(im_start_rgb.shape)
    #print(im_start[66,:,13,20])
    im_start_rgb[:,0,:,:]=im_start[:,2,:,:]
    im_start_rgb[:,1,:,:]=im_start[:,1,:,:]
    im_start_rgb[:,2,:,:]=im_start[:,0,:,:]
    #print(im_start_rgb[66,:,13,20])

    im_goal=im_ar[:,3:,:,:]
    #print(im_goal.shape)
    im_goal_rgb=np.zeros(im_goal.shape)
    #print(im_goal_rgb.shape)
    #print(im_goal[66,:,13,20])
    im_goal_rgb[:,0,:,:]=im_goal[:,2,:,:]
    im_goal_rgb[:,1,:,:]=im_goal[:,1,:,:]
    im_goal_rgb[:,2,:,:]=im_goal[:,0,:,:]
    #print(im_goal_rgb[66,:,13,20])
    #print(im_goal.shape)
    
    con_image=np.concatenate((np.array(im_start_rgb),np.array(im_goal_rgb)), axis=2)
    #print(con_image[66,:,13,20])
    #print(con_image[66,:,13+64,20])
    #print(con_image.shape)
    result = np.transpose(con_image, (0, 2, 3, 1))
    #print(result.shape)
    #print(result[66,13,20,:])
    #print(result[66,13+64,20,:])
    #print("===================================================")
    return result
    
train_im_tl=concate_tl(train_images)
test_im_tl=concate_tl(test_images)
train_im_mix_tl=concate_tl(train_images_mix)
test_im_mix_tl=concate_tl(test_images_mix)

#print(train_flows[12,1,36,48])
#for i in range(64):
#    for j in range(64):
#        if train_flows[12,1,i,j]!=0:
#            print(str(i)+str(j))
train_flows_r = np.transpose(train_flows, (0, 2, 3, 1))
#print(train_flows_r[12,36,48, 1])
#print(train_flows_r.shape)
test_flows_r = np.transpose(test_flows, (0, 2, 3, 1))
#print(test_flows_r.shape)

train_flows_mix_r = np.transpose(train_flows_mix, (0, 2, 3, 1))
#print(train_flows_mix_r.shape)
test_flows_mix_r = np.transpose(test_flows_mix, (0, 2, 3, 1))
#print(test_flows_mix_r.shape)

I developed and trained many models. The model that includes the first 2 conv layers of the VGG-16 model pre-trained on ImageNet with the VGG layers' weights fixed. One deconv layer is attached after the VGG to increase the number of neurons to meet the requirement for the output. After that, I added 6 conv layers and one regression layer as output layer.
I tried keeping different number of VGG conv layers ranging from first 2 to all of them. And I also tried adding 0 to 6 conv layers from my customized network after that. I changed the optimizers as well including SGD, Adam and Adagrad, but Adam with learning rate 0.001 is obviously the best. Additionally, the batch size is 10. The max number of epochs for training is 100.
I did the experiments on the rotation dataset and the mixed dataset.
Most of the time, the training MSE stops at roughly 0.64 ish in rotation dataset. 
In a few cases, the training MSE goes down gradually to 0.2898. However, the Mean Squared Error for validation always stops at roughly 0.64ish in rotation dataset.



For this question adapt
VGG-16 pre-trained on ImageNet for the task. The pre-trained network is available from
tf.keras.applications.vgg16. You want to suitably remove some layers (importing with
include top=False is a good start but less layers are likely suficient and will run much faster)
and add a regression layer (same or similar as above) at the output. You will need to train the
new layers with the weights of the existing VGG layers fixed. Once you have a working regressor
for the task, try to improve the 
flow results by training some layers a bit more (typically the
higher-level) layers. Train and test your model als with the rotational and the mixed dataset. On
which dataset does your network perform better.

Here are some examples:
The model with first 6 conv layers with all parameters fixed, followed by a deconv layer and an output layer.
Epoch 1/100
1655/1655 [==============================] - 4s 2ms/sample - loss: 1.3269 - mean_squared_error: 1.3269 - val_loss: 0.8023 - val_mean_squared_error: 0.8023
Epoch 2/100
1655/1655 [==============================] - 2s 1ms/sample - loss: 0.7129 - mean_squared_error: 0.7129 - val_loss: 0.7292 - val_mean_squared_error: 0.7292
...

Epoch 100/100
1655/1655 [==============================] - 2s 1ms/sample - loss: 0.6328 - mean_squared_error: 0.6328 - val_loss: 0.6795 - val_mean_squared_error: 0.6795


The model with first 9 conv layers with all parameters fixed, followed by a deconv layer, 2 conv layers with 128 neurons each and an output layer.
Epoch 100/100
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.2898 - mean_squared_error: 0.2898 - val_loss: 0.6389 - val_mean_squared_error: 0.6389



In [16]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, MaxPooling3D, UpSampling3D,Conv2DTranspose, Conv3DTranspose
from tensorflow.keras.models import Model

#For the RTX 2060 GPU to work in Tensorflow-gpu 2.0, setting the memory growth.
import tensorflow.keras
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

#USE ALL tf.keras
tmodel = tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(128, 64, 3), pooling=None)
print(tmodel.summary())

#Remove the last # layers
num_del=15
for i in range(num_del):
    tmodel._layers.pop()

#unfreeze the last n layers
num_unf=0
print(tmodel.summary())
# Freeze all the VGG layers.
for layer in tmodel.layers[:19-num_del-num_unf]:
    layer.trainable = False
print(tmodel.summary())

#Use Deconv Layer to increase the number of neurons
layer_deconv=Conv2DTranspose(64, kernel_size=(1, 1),strides=(1, 2), data_format='channels_last')

#Add conv layers after VGG model
layer_conv=tf.keras.layers.Conv2D(64, (1, 1), activation='relu', padding='same', data_format='channels_last')
layer_conv2=tf.keras.layers.Conv2D(64, (5, 5), activation='relu', padding='same', data_format='channels_last')
layer_conv3=tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_last')
layer_conv4=tf.keras.layers.Conv2D(64, (5, 5), activation='relu', padding='same', data_format='channels_last')
layer_conv5=tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_last')
layer_conv6=tf.keras.layers.Conv2D(64, (5, 5), activation='relu', padding='same', data_format='channels_last')

# creating the model with a regression output layer
layer_reg=tf.keras.layers.Conv2D(2, (3, 3), activation='linear', padding='same', data_format='channels_last')
#Max Pooling layer
#layer_pool=tf.keras.layers.MaxPooling2D(pool_size=(2,1))

tmodel_one_reg = tf.keras.Sequential()
#tmodel_one_reg.build(input_shape=(128, 64, 3))
for layer in tmodel.layers:
    tmodel_one_reg.add(layer)
tmodel_one_reg.add(layer_deconv)
tmodel_one_reg.add(layer_conv)
tmodel_one_reg.add(layer_conv2)
tmodel_one_reg.add(layer_conv3)
tmodel_one_reg.add(layer_conv4)
tmodel_one_reg.add(layer_conv5)
tmodel_one_reg.add(layer_conv6)
tmodel_one_reg.add(layer_reg)
#tmodel_one_reg.add(layer_pool)


print(tmodel_one_reg.summary())
tmodel_one_reg1=tmodel_one_reg #Create the model for mixed dataset

#Optimizers
sgd = tf.keras.optimizers.SGD(lr=0.001)
adam = tf.keras.optimizers.Adam(lr=0.001)
nadam = tf.keras.optimizers.Nadam(lr=0.001)
adagrad = tf.keras.optimizers.Adagrad(lr=0.001)

tmodel_one_reg.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer='adam',
              metrics=['mean_squared_error'])

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 128, 64, 3)]      0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 64, 64)       1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 64, 64)       36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 16, 128)       0     

In [7]:
batchSize = 10
nEpochs = 20 #100

#history = model.fit(train_img_rot, train_flows, batch_size=batchSize, epochs=nEpochs, verbose=1,validation_split=0.11)
history = tmodel_one_reg.fit(train_im_tl, train_flows_r, batch_size=batchSize, epochs=nEpochs, verbose=1,validation_split=0.11)

Train on 1655 samples, validate on 205 samples
Epoch 1/20
1655/1655 [==============================] - 5s 3ms/sample - loss: 0.6499 - mean_squared_error: 0.6499 - val_loss: 0.6819 - val_mean_squared_error: 0.6819
Epoch 2/20
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.6401 - mean_squared_error: 0.6401 - val_loss: 0.6812 - val_mean_squared_error: 0.6812
Epoch 3/20
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.6385 - mean_squared_error: 0.6385 - val_loss: 0.6800 - val_mean_squared_error: 0.6800
Epoch 4/20
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.6360 - mean_squared_error: 0.6360 - val_loss: 0.6743 - val_mean_squared_error: 0.6743
Epoch 5/20
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.6329 - mean_squared_error: 0.6329 - val_loss: 0.6741 - val_mean_squared_error: 0.6741
Epoch 6/20
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.6282 - mean_squared_error: 0.6282 - val_los

Using the transfer learning model, the Mean Squared Error for Rotation Test Set is: 0.6088. While using the customized model, the MSE for test set is only 0.0056, which is significantly lower than that of TL model by 0.6032.
Apparently the performance of this Transfer Learning model is worse than the customized model. I think the reason is that the fixed weights from VGG layers are learned to perform classification task, rather than regression task. In VGG-16, the goal is to do object identification in images. However, our task now is to find optical flows, which is the movement of the object. Neural Network is very sensitive to the hyperparameters. Add that these are two different tasks so that the parameters including weights and biases should be very different. Therefore, the weights and biases from VGG network may not be that useful for the regression task that find the optimal flows.

In [8]:
from sklearn.metrics import mean_squared_error

test_flow_result = tmodel_one_reg.predict(test_im_tl)
print(test_flows_r.shape)
#Reshape to make the flow array two dimensional so that it can use sklearn MSE function
test_flowsa=test_flows_r.reshape(test_flows_r.shape[0],8192) #8192=64*64*2
print(test_flowsa.shape)
print(test_flow_result.shape)
test_flow_resulta=test_flow_result.reshape(test_flows_r.shape[0],8192)
print(test_flow_resulta.shape)

mse=mean_squared_error(test_flowsa, test_flow_resulta)
#msle=mean_squared_log_error(train_flowsa, train_flow_resulta)
#Mean Squared Logarithmic Error cannot be used when targets contain negative values
print("Mean Squared Error for Test Set is: %.4f"%mse)
#print(str(msle))
#print(str(medae))

(444, 64, 64, 2)
(444, 8192)
(444, 64, 64, 2)
(444, 8192)
Mean Squared Error for Test Set is: 0.6175


Then I used that TL model with Mixed dataset, the training MSE stops at around 0.62, while the vaidation MSE stops at 0.6086. This shows that the TL model get stuck at local minimum and could not continue learning even though I increase the number of epochs.

In [17]:
batchSize = 10
nEpochs = 6

#history = model.fit(train_img_rot, train_flows, batch_size=batchSize, epochs=nEpochs, verbose=1,validation_split=0.11)
history = tmodel_one_reg1.fit(train_im_mix_tl, train_flows_mix_r, batch_size=batchSize, epochs=nEpochs, verbose=1,validation_split=0.11)

Train on 1655 samples, validate on 205 samples
Epoch 1/6
1655/1655 [==============================] - 5s 3ms/sample - loss: 0.6517 - mean_squared_error: 0.6517 - val_loss: 0.6085 - val_mean_squared_error: 0.6085
Epoch 2/6
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.6224 - mean_squared_error: 0.6224 - val_loss: 0.6769 - val_mean_squared_error: 0.6769
Epoch 3/6
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.6236 - mean_squared_error: 0.6236 - val_loss: 0.6086 - val_mean_squared_error: 0.6086
Epoch 4/6
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.6234 - mean_squared_error: 0.6234 - val_loss: 0.6086 - val_mean_squared_error: 0.6086
Epoch 5/6
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.6234 - mean_squared_error: 0.6234 - val_loss: 0.6086 - val_mean_squared_error: 0.6086
Epoch 6/6
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.6234 - mean_squared_error: 0.6234 - val_loss: 0.6

And then I tested the TL model on mixed dataset, and got the MSE of 0.5995. This is slightly lower than the model on the rotation dataset which is 0.6088. I think the reason why it does not perform as well as the customized model is that the weights from VGG are fixed but the task of VGG is classification rather than regression.

In [18]:
from sklearn.metrics import mean_squared_error

test_flow_mix_result = tmodel_one_reg1.predict(test_im_mix_tl)
print(test_flows_mix_r.shape)
#Reshape to make the flow array two dimensional so that it can use sklearn MSE function
test_mix_flowsa=test_flows_mix_r.reshape(test_flows_mix_r.shape[0],8192) #8192=64*64*2
print(test_flowsa.shape)
print(test_flow_result.shape)
test_flow_mix_resulta=test_flow_mix_result.reshape(test_flow_mix_result.shape[0],8192)
print(test_flow_mix_resulta.shape)

mse=mean_squared_error(test_mix_flowsa, test_flow_mix_resulta)
#msle=mean_squared_log_error(train_flowsa, train_flow_resulta)
#Mean Squared Logarithmic Error cannot be used when targets contain negative values
print("Mean Squared Error for Test Set is: %.4f"%mse)
#print(str(msle))
#print(str(medae))

(444, 64, 64, 2)
(444, 8192)
(444, 64, 64, 2)
(444, 8192)
Mean Squared Error for Test Set is: 0.5995


Then I trained the customized model with the mixed dataset, and the training MSE is 0.0127, while the validation MSE is a bit higher, 0.0222. It can be seen that the MSE for both training and validation are decreasing gradually, which means that of I increase the number of epochs, the result could be even better.

In [19]:
batchSize = 10
nEpochs = 20 #100

#history = model.fit(train_img_rot, train_flows, batch_size=batchSize, epochs=nEpochs, verbose=1,validation_split=0.11)
history = model1.fit(train_images_mix, train_flows_mix, batch_size=batchSize, epochs=nEpochs, verbose=1,validation_split=0.11)

Train on 1655 samples, validate on 205 samples
Epoch 1/20
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.0258 - mean_squared_error: 0.0258 - val_loss: 0.0271 - val_mean_squared_error: 0.0271
Epoch 2/20
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.0255 - mean_squared_error: 0.0255 - val_loss: 0.0290 - val_mean_squared_error: 0.0290
Epoch 3/20
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.0211 - mean_squared_error: 0.0211 - val_loss: 0.0306 - val_mean_squared_error: 0.0306
Epoch 4/20
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.0206 - mean_squared_error: 0.0206 - val_loss: 0.0263 - val_mean_squared_error: 0.0263
Epoch 5/20
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.0223 - mean_squared_error: 0.0223 - val_loss: 0.0248 - val_mean_squared_error: 0.0248
Epoch 6/20
1655/1655 [==============================] - 4s 2ms/sample - loss: 0.0188 - mean_squared_error: 0.0188 - val_los

I tested the customized model on mixed dataset. The Mean Squared Error is 0.0208, which is considerably higher than the MSE on rotation dataset, 0.0056 by 0.0152. However, the customized model is apparently better than the TL model, while the MSE for TL model is as high as 0.5995. 

In [20]:
from sklearn.metrics import mean_squared_error

test_flow_mix_result = model1.predict(test_images_mix)
#Reshape to make the flow array two dimensional so that it can use sklearn MSE function
test_flowsa_mix=test_flows_mix.reshape(test_flows_mix.shape[0],8192) #8192=2*64*64
print(test_flows_mix.shape[0])
test_flow_mix_resulta=test_flow_mix_result.reshape(test_flows_mix.shape[0],8192)

mse=mean_squared_error(test_flowsa_mix, test_flow_mix_resulta)
#msle=mean_squared_log_error(train_flowsa, train_flow_resulta)
#Mean Squared Logarithmic Error cannot be used when targets contain negative values
print("Mean Squared Error for Test Set is: %.4f"%mse)
#print(str(msle))
#print(str(medae))

444
Mean Squared Error for Test Set is: 0.0208


In conclusion, the performance of the customized model for both rotation dataset and mixed dataset are much better than the Transfer Learning model. And the performance of the customized model for rotation dataset is better than mixed dataset. In contrast, the performance of the Transfer learning model for rotation dataset is worse than mixed dataset.
I believe the reason for the overall better performance of customized model is that the fixed weights from VGG layers are learned to perform classification task, which is different from our task now to find optical flows. Therefore, the weights and biases from VGG network may not be that useful for the regression task that find the optimal flows. As a result, the local minimum rather than the global minimum could be reached.
And the reason for the fact that customized model performing better for rotation dataset than mixed dataset is that the problem is simpler in the rotation dataset in which the images have in-plane rotations and translations only. However, the problem is more difficult when there is a mixture of rotations, shearing and stretching combined with translations in the mixed dataset.
Last but not the least, the weights in VGG model could be learnt to classify images, which are more complicated problems or more higher-level features. That's the reason why Transfer Learning model perform better for mixed dataset rather than rotation dataset, because the former is more difficult and more complicated.